In [1]:
# Near field
# 
import meep as mp
import numpy as np

# --- Simulation parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10  # pixels/μm

pml_thickness = 1.0  # μm
padding = 2.0        # distance from source to PML
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

def simulate_full_fields(obs_points):
    if obs_points.ndim == 1:
        obs_points = obs_points[np.newaxis, :]

    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    dft_obj = sim.add_dft_fields(
        [mp.Ex, mp.Ey, mp.Ez, mp.Hx, mp.Hy, mp.Hz],
        [frequency],
        center=mp.Vector3(),
        size=cell_size
    )

    sim.run(until=100)

    Ex = sim.get_dft_array(dft_obj, mp.Ex, 0)
    Ey = sim.get_dft_array(dft_obj, mp.Ey, 0)
    Ez = sim.get_dft_array(dft_obj, mp.Ez, 0)
    Hx = sim.get_dft_array(dft_obj, mp.Hx, 0)
    Hy = sim.get_dft_array(dft_obj, mp.Hy, 0)
    Hz = sim.get_dft_array(dft_obj, mp.Hz, 0)

    grid_size = sim.get_array_metadata()  # ✅ this returns (nx, ny, nz)

    delta = 1.0 / resolution
    origin = -0.5 * np.array([cell_dim, cell_dim, cell_dim])

    def to_index(vec):
        return tuple(((vec - origin) / delta).astype(int))

    E_all = []
    H_all = []

    for pt in obs_points:
        idx = to_index(np.array(pt))
        E = np.array([Ex[idx], Ey[idx], Ez[idx]])
        H = np.array([Hx[idx], Hy[idx], Hz[idx]])
        E_all.append(E)
        H_all.append(H)

    return np.array(E_all), np.array(H_all)

# --- Example usage ---
points = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

E, H = simulate_full_fields(points)

for i, pt in enumerate(points):
    print(f"\nPoint {i} at {pt}:")
    print(f"  E = {E[i]}")
    print(f" |E| = {np.linalg.norm(E[i])}")
    print(f"  H = {H[i]}")
    print(f" |H| = {np.linalg.norm(H[i])}")
    print(f" |E|/|H| = {np.linalg.norm(E[i]) / np.linalg.norm(H[i])}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.00196099 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.688116 s
-----------
Meep progress: 6.5/100.0 = 6.5% done in 4.0s, 57.8s to go
on time step 130 (time=6.5), 0.0309018 s/step
Meep progress: 16.45/100.0 = 16.4% done in 8.0s, 40.7s to go
on time step 329 (time=16.45), 0.0201252 s/step
Meep progress: 29.55/100.0 = 29.6% done in 12.0s, 28.7s to go
on time step 591 (time=29.55), 0.0152835 s/step
Meep progress: 44.150000000000006/100.0 = 44.2% done in 16.0s, 20.3s to go
on time step 883 (time=44.15), 0.0137153 s/step
Meep progress: 56.95/100.0 = 57.0% done in 20.0s, 15.1s to go
on time step 1139 (time=56.95), 0.0156298 s/step
Meep progress: 69.55/100.0 = 69.5% done in 24.0s, 10.5s to go
on time step 1391 (time=69.55), 0.0158814 s/step
Meep progress: 83.4/100.0 = 83.4% done in 28.1s, 5.6s to go
on time step 1668 (time=83.4), 0.0144836 s/step
Meep progress: 96.

In [2]:
# Far field

import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Define a near-field box around the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # fcen, df, nfreq
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2 * pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2 * pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2 * pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    # Calculate far-field E and H
    ff_fields = sim.get_farfield(nearfield_box, mp.Vector3(*far_point))
    E = np.array(ff_fields[0:3])
    H = np.array(ff_fields[3:6])

    return E, H

# --- Example usage ---
far_point = [3.0*wavelength, 2.0*wavelength, 4.0*wavelength]
E_far, H_far = simulate_far_field(far_point)

print(f"\nFar-field at {far_point} μm:")
print(f"  E = {E_far}")
print(f" |E| = {np.linalg.norm(E_far)}")
print(f"  H = {H_far}")
print(f" |H| = {np.linalg.norm(H_far)}")
print(f" |E|/|H| = {np.linalg.norm(E_far) / np.linalg.norm(H_far)} (should ≈ 376.73 in vacuum)")
print("E and H are not correctly scaled")


-----------
Initializing structure...
time for choose_chunkdivision = 0.000332832 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.368658 s
-----------
Meep progress: 22.85/100.0 = 22.9% done in 4.0s, 13.5s to go
on time step 457 (time=22.85), 0.00876059 s/step
Meep progress: 51.85/100.0 = 51.9% done in 8.0s, 7.4s to go
on time step 1037 (time=51.85), 0.00689793 s/step
Meep progress: 81.5/100.0 = 81.5% done in 12.0s, 2.7s to go
on time step 1630 (time=81.5), 0.00674687 s/step
run 0 finished at t = 100.0 (2000 timesteps)

Far-field at [0.9750000000000001, 0.65, 1.3] μm:
  E = [ 75.8210864  -1.20650233j  62.2304824  +2.28936927j
 -86.27545244+10.33301761j]
 |E| = 131.06636444408767
  H = [-78.38601172+2.98923827j 103.88474406-6.54213944j
   6.19346391+2.14807902j]
 |H| = 130.5033151119549
 |E|/|H| = 1.0043144446687025 (should ≈ 376.73 in vacuum)
E and H are not correctly scaled


In [3]:
E_far

array([ 75.8210864  -1.20650233j,  62.2304824  +2.28936927j,
       -86.27545244+10.33301761j])

In [4]:
# Calculating H using impedence in free space

# Compute radial direction
r_hat = far_point / np.linalg.norm(far_point)


# Impedance of free space
eta0 = 376.730

# Compute far-field H
H_far = np.cross(r_hat, E_far) / eta0

# Display results
print("E_far:", E_far)
print("H_far:", H_far)
print("|E|   =", np.linalg.norm(E_far))
print("|H|   =", np.linalg.norm(H_far))
print("|E|/|H| =", np.linalg.norm(E_far) / np.linalg.norm(H_far))


E_far: [ 75.8210864  -1.20650233j  62.2304824  +2.28936927j
 -86.27545244+10.33301761j]
H_far: [-0.20774969+0.00567272j  0.27707201-0.01765866j  0.01727626+0.00457479j]
|E|   = 131.06636444408767
|H|   = 0.3472639290473506
|E|/|H| = 377.425794851893


In [5]:
import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

# --- Far-field Simulation Function ---
def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Near-to-far field monitor surrounding the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # single frequency
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2*pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2*pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2*pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    r = mp.Vector3(*far_point)
    ff_fields = sim.get_farfield(nearfield_box, r)

    # Split into E and H (6 values returned per frequency)
    ff_E = np.array(ff_fields[:3])
    ff_H = np.array(ff_fields[3:])
    
    return ff_E, ff_H

# --- Main Execution ---
if __name__ == "__main__":
    far_point = [10.0, 0.0, 0.0]  # μm
    eta0 = 376.730  # Vacuum impedance (Ohm)

    E_far, H_meep = simulate_far_field(far_point)
    r_hat = np.array(far_point) / np.linalg.norm(far_point)
    H_calc = np.cross(r_hat, E_far) / eta0

    # --- Print Results ---
    print(f"\n📍 Far-field at {far_point} μm:")
    print("E_far     =", E_far)
    print("H_meep    =", H_meep)
    print("H_calc    =", H_calc)

    print("\n--- Norms ---")
    print(f"|E|        = {np.linalg.norm(E_far):.6f}")
    print(f"|H_meep|   = {np.linalg.norm(H_meep):.6f}")
    print(f"|H_calc|   = {np.linalg.norm(H_calc):.6f}")

    print("\n--- Ratios ---")
    print(f"|E|/|H_meep|  = {np.linalg.norm(E_far)/np.linalg.norm(H_meep):.6f} (should ≈ η = {eta0})")
    print(f"|E|/|H_calc|  = {np.linalg.norm(E_far)/np.linalg.norm(H_calc):.6f} (should = η)")

    print("\n--- Difference ---")
    print(f"ΔH = ||H_meep - H_calc|| = {np.linalg.norm(H_meep - H_calc):.6e}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.000442028 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.380935 s
-----------
Meep progress: 24.900000000000002/100.0 = 24.9% done in 4.0s, 12.1s to go
on time step 498 (time=24.9), 0.00804246 s/step
Meep progress: 56.150000000000006/100.0 = 56.2% done in 8.0s, 6.3s to go
on time step 1123 (time=56.15), 0.00640066 s/step
Meep progress: 86.55000000000001/100.0 = 86.6% done in 12.0s, 1.9s to go
on time step 1731 (time=86.55), 0.00658184 s/step
run 0 finished at t = 100.0 (2000 timesteps)

📍 Far-field at [10.0, 0.0, 0.0] μm:
E_far     = [ 1.35470016e-01+6.86606701e-02j -4.42973837e-15-1.60798024e-15j
  1.91613003e+01-2.23774235e+01j]
H_meep    = [-6.14537156e-02-1.84313936e-01j -1.91615307e+01+2.23777660e+01j
 -3.11590218e-15-1.08399888e-15j]
H_calc    = [ 0.00000000e+00+0.00000000e+00j -5.08621568e-02+5.93991014e-02j
 -1.17583903e-17-4.26825642e-18j]

--- Norms